In [3]:
#### Solving MaxCut, problem instances are from
#### https://biqmac.aau.at/biqmaclib.html#ref
#### http://bqp.cs.uni-bonn.de/library/html/instances.html

using ITensors, ITensorMPS
# require LinearAlgebra for matrix manipulations 
using LinearAlgebra
# for saving and reading MPS
using HDF5
# for random numbers etc.
using Statistics
using Random 
using DelimitedFiles

### input: instance file name
function maxcut(fn)
    println("Reading MaxCut file name: ", fn)
    dlm, header = readdlm(fn, Int32, header=true) 

    NV = parse(Int, header[1])
    M = NV ### dimension of the qubo matrices
    NE = parse(Int, header[2])
 
    println("# of vertices = ",NV," , # of edges = ", NE)
    
    # create a Q matrix, array initialized to zero 
    Q = zeros(Float64, M, M)
    
    #loop over the edges
    for k = 1:NE
        xi = dlm[k,1] ## vertex 1
        xj = dlm[k,2] ## vertex 2
    
        wij = dlm[k,3] 
        Q[xi, xj] += 2*wij
        Q[xi, xi] -= wij
        Q[xj, xj] -= wij
    end
    Q = triu(Q) + transpose(tril(Q, -1))

    en_shift = 0.25*(tr(Q) + sum(Q))
    
    ## compute the onsite magnetic fields for reduced qubo, to be used for h1, the problem hamiltonian
    ha = zeros(M)
    for i in eachindex(ha)
        slice1 = Q[i,i+1:M]
        slice2 = Q[1:i-1,i]
        ha[i] = Q[i,i] + 0.5*( sum(slice1) + sum(slice2) ) 
    end    
    println("Max of ha ",maximum(ha)," Min of ha ", minimum(ha))
    
    ## important note: with no conserved QN, otherwise one cannot use h2, the transverse hamiltonian
    sites = siteinds("S=1/2",M)   
    
    ## set up h1, the problem Hamiltonian, as OpSum 
    h1 = OpSum()
    for i=1:M
        h1 += (ha[i], "Sz", i)
        for j in i+1:M
            h1 += (Q[i,j], "Sz", i, "Sz", j)  
        end
    end

    println("The energy shift is ", en_shift)
    println("Max and Min of the diagonal of H: ", maximum(ha)," , ", minimum(ha))
    
    ## initial state is a product state, all spins point along x, |+> or |->
    psi = random_mps(sites; linkdims=7) 
    #psi = MPS(sites, ["-" for n in 1:M] )
    println("Starting from a random MPS:")

    ### these depend on the problem instance, 
    numsteps = 5
    bond_dimension = 30
    numsweeps = 10
    hx_const = 1
    hx_random_scale = 0
    
    del = 1.0/numsteps
    
    en_list = zeros(Float64, numsteps)
    sx_list = zeros(Float64, numsteps)
    sz_list = zeros(Float64, numsteps)
    idx = 1
    
    for sp in range(start=1-del,step=-del,length=numsteps) 
        println("step sp  = ", sp)

        ## set up the transverse Hamiltonian with added randomness
        rd = hx_random_scale*randn(M)
        h2 = OpSum()
        for i=1:M
            h2 += (hx_const+rd[i], "Sx", i) 
        end
    
        H = MPO( (1.0-sp)*h1 + sp*h2, sites) ### used in most
        
        # eigsolve_krylovdim=7, noise=1E-5 used for many 
        if idx <10
            energy,psi = dmrg(H,psi;nsweeps=numsweeps,maxdim=bond_dimension,noise=0.003) 
        else 
            ### winding down D manually to reach the product state, usually not needed 
            energy,psi = dmrg(H,psi;nsweeps=numsweeps,maxdim=[bond_dimension,20,10,8,6,5,4,3,2,1],noise=1e-5)
        end 
        
        en_list[idx] = energy + en_shift

        #sigz = expect(psi, "Sz")
        #sz_list[idx] = sum(sigz)
        #println("sum Z = ", sum(sigz))
        #println("Z = ", sigz)
        #sigx = expect(psi, "Sx")
        #sx_list[idx] = sum(sigx)
        # println("sum X = ", sum(sigx))
        # println("sum X = ", sigx)

        idx = idx + 1
    end
    
    ### call DMRG again to get energy, winding down the bond dimension, may not be needed
    #H = MPO( h1, sites)
    #energy,psi = dmrg(H,psi;nsweeps=10,maxdim=[10,9,8,7,6,5,4,3,2,1],noise=0.0003) # was 60
    
    println("EN: ", en_list)
    #println("Sz: ", sz_list)
    #println("Sx: ", sx_list)
    
    ### compute the expectation value of Sz
    Z = expect(psi, "Sz")
    #println("sum Z = ", sum(Z))

    bn = zeros(Int8,M)
    for i in eachindex(Z)
        if abs(Z[i]-0.5) < 0.5  ## if closer to 1/2; was < 0.1 which seems quite harsh
            bn[i]=1 ## Recall: spin up == 1, spin down == 0
        end
    end

    println("Subset contains ", sum(bn), " vertices out of ",NV,":")
    print("{ ")
    for i in eachindex(bn)
        if bn[i] > 0.5
            print(i," ")
        end
    end
    print("}\n")
    
    cost = 0
    for k = 1:NE
        xi = dlm[k,1] ## vertex 1
        xj = dlm[k,2] ## vertex 2
    
        wij = dlm[k,3] 
        v1 = bn[xi]
        v2 = bn[xj]
        cost += wij*(v1-v2)*(v1-v2)
    end
    println("The maximal solution value is ",cost)

    return
end

# This one will run very fast
maxcut("./rudy_all/pm1s_80.0")

Reading MaxCut file name: ./rudy_all/pm1s_80.0
# of vertices = 80 , # of edges = 316
Max of ha 0.0 Min of ha 0.0
The energy shift is 4.0
Max and Min of the diagonal of H: 0.0 , 0.0
Starting from a random MPS:
step sp  = 0.8
After sweep 1 energy=-33.40078654983073  maxlinkdim=30 maxerr=1.27E-03 time=0.923
After sweep 2 energy=-33.458466985676196  maxlinkdim=30 maxerr=9.14E-04 time=1.629
After sweep 3 energy=-33.473404673002875  maxlinkdim=30 maxerr=9.63E-04 time=1.465
After sweep 4 energy=-33.53860095010228  maxlinkdim=30 maxerr=9.43E-04 time=1.608
After sweep 5 energy=-33.71002993600347  maxlinkdim=30 maxerr=6.72E-04 time=1.348
After sweep 6 energy=-33.78276853355412  maxlinkdim=30 maxerr=6.61E-04 time=1.711
After sweep 7 energy=-33.79824890464087  maxlinkdim=30 maxerr=6.69E-04 time=1.354
After sweep 8 energy=-33.803890440959385  maxlinkdim=30 maxerr=6.74E-04 time=1.522
After sweep 9 energy=-33.806998106727754  maxlinkdim=30 maxerr=6.75E-04 time=1.374
After sweep 10 energy=-33.80875785